In [1]:
sc

org.apache.spark.SparkContext@2efd4928

In [2]:
spark

org.apache.spark.sql.SparkSession@328f6ec3

### Read File Into DataFrame:

In [3]:
// please change the path
val transactionsDF = spark.read.format("csv").option("sep", ",")
.option("header", "false")
.load("/home/leo/IdeaProjects/DS503/Project3/input/Transactions")
val newColumns = Seq("TransID","CustID","TransTotal","TransNumItems","TransDesc")
val temp1 = transactionsDF.toDF(newColumns:_*)
// change column type of the dataframe
val TransactionsDF = temp1.withColumn("TransID", temp1("TransID").cast(org.apache.spark.sql.types.IntegerType))
.withColumn("CustID", temp1("CustID").cast(org.apache.spark.sql.types.IntegerType))
.withColumn("TransTotal", temp1("TransTotal").cast(org.apache.spark.sql.types.FloatType))
.withColumn("TransNumItems", temp1("TransNumItems").cast(org.apache.spark.sql.types.IntegerType))


transactionsDF = [_c0: string, _c1: string ... 3 more fields]
newColumns = List(TransID, CustID, TransTotal, TransNumItems, TransDesc)
temp1 = [TransID: string, CustID: string ... 3 more fields]
TransactionsDF = [TransID: int, CustID: int ... 3 more fields]


[TransID: int, CustID: int ... 3 more fields]

In [4]:
// have a glance at the dataframe contents
TransactionsDF.show

+-------+------+----------+-------------+--------------------+
|TransID|CustID|TransTotal|TransNumItems|           TransDesc|
+-------+------+----------+-------------+--------------------+
|      1|  2389|     553.9|            9|rodcvpyykzsyxcmfu...|
|      2|  3878|    844.42|            8|arxtfzjjtjtzixqla...|
|      3|   934|    448.95|           10|mguwsuolmcjowxgqw...|
|      4|  3965|    759.41|           10|fpqvaqdbhbkncasos...|
|      5|   347|    383.84|            1|ybupgovpohccpounl...|
|      6|  4447|    703.49|            9|gqlhoktdrftllpkei...|
|      7|  1283|    299.19|            3|lenqjmcthfmoqcuxo...|
|      8|   326|     574.2|            4|kpvfrbtrcddqkrtnl...|
|      9|  3344|    300.12|            9|bdiveswzqnzxyzmqi...|
|     10|  1771|    417.52|            6|iigaecdlsrxawqdlr...|
|     11|  1346|    203.95|           10|rvxdgdvpokltpzjng...|
|     12|  3009|    211.65|            8|zdlxoifokcdofnitq...|
|     13|  3881|    904.15|            3|gnwdwwqnxgcnca

In [5]:
// have a glance at the TransactionDataFrame's structure
TransactionsDF.printSchema

root
 |-- TransID: integer (nullable = true)
 |-- CustID: integer (nullable = true)
 |-- TransTotal: float (nullable = true)
 |-- TransNumItems: integer (nullable = true)
 |-- TransDesc: string (nullable = true)



### (1) T1:

In [6]:
val result1 = TransactionsDF.filter("TransTotal >= 200")
result1.show

+-------+------+----------+-------------+--------------------+
|TransID|CustID|TransTotal|TransNumItems|           TransDesc|
+-------+------+----------+-------------+--------------------+
|      1|  2389|     553.9|            9|rodcvpyykzsyxcmfu...|
|      2|  3878|    844.42|            8|arxtfzjjtjtzixqla...|
|      3|   934|    448.95|           10|mguwsuolmcjowxgqw...|
|      4|  3965|    759.41|           10|fpqvaqdbhbkncasos...|
|      5|   347|    383.84|            1|ybupgovpohccpounl...|
|      6|  4447|    703.49|            9|gqlhoktdrftllpkei...|
|      7|  1283|    299.19|            3|lenqjmcthfmoqcuxo...|
|      8|   326|     574.2|            4|kpvfrbtrcddqkrtnl...|
|      9|  3344|    300.12|            9|bdiveswzqnzxyzmqi...|
|     10|  1771|    417.52|            6|iigaecdlsrxawqdlr...|
|     11|  1346|    203.95|           10|rvxdgdvpokltpzjng...|
|     12|  3009|    211.65|            8|zdlxoifokcdofnitq...|
|     13|  3881|    904.15|            3|gnwdwwqnxgcnca

result1 = [TransID: int, CustID: int ... 3 more fields]


[TransID: int, CustID: int ... 3 more fields]

### (2) T2:

In [7]:
result1.createOrReplaceTempView("Transactions")
val summary1=spark.sql("Select TransNumItems,sum(TransTotal),avg(TransTotal),min(TransTotal),max(TransTotal) from Transactions group by TransNumItems")

summary1 = [TransNumItems: int, sum(TransTotal): double ... 3 more fields]


[TransNumItems: int, sum(TransTotal): double ... 3 more fields]

### (3):

In [8]:
summary1.show

+-------------+--------------------+-----------------+---------------+---------------+
|TransNumItems|     sum(TransTotal)|  avg(TransTotal)|min(TransTotal)|max(TransTotal)|
+-------------+--------------------+-----------------+---------------+---------------+
|            1| 2.411747113102722E7|599.5642294848284|         200.01|         999.98|
|            6|2.4228478623138428E7|600.3835614704108|         200.02|         1000.0|
|            3| 2.411609182826233E7|597.3321731915471|          200.0|         999.94|
|            5|2.4356179616455078E7|598.6672799246652|         200.01|         1000.0|
|            9|2.4168764228775024E7|598.7406289643518|         200.04|         999.98|
|            4| 2.426231051916504E7|599.5282937350821|         200.01|         1000.0|
|            8| 2.415729585119629E7|597.5978589747746|         200.02|         1000.0|
|            7|2.4205503997314453E7|602.0670579373807|         200.02|         999.98|
|           10|2.4460900304901123E7|602.173

### (4) T3:

In [9]:
val summary2=spark.sql("SELECT CustID,COUNT(CustID) as count1 FROM Transactions Group By CustID")

summary2 = [CustID: int, count1: bigint]


[CustID: int, count1: bigint]

### (5) T4:

In [10]:
val result2 = TransactionsDF.filter("TransTotal < 600")

result2 = [TransID: int, CustID: int ... 3 more fields]


[TransID: int, CustID: int ... 3 more fields]

### (6) T5:

In [11]:
result2.createOrReplaceTempView("Transactions2")
val summary3=spark.sql("SELECT CustID,COUNT(CustID) as count2 FROM Transactions2 Group By CustID")

summary3 = [CustID: int, count2: bigint]


[CustID: int, count2: bigint]

### (7) T6:

In [12]:
summary2.createOrReplaceTempView("TC1")
summary3.createOrReplaceTempView("TC2")
val summary4=spark.sql("SELECT TC1.CustID FROM TC1,TC2 WHERE TC1.CustID=Tc2.CustID and TC1.count1>(5*TC2.count2)")

summary4 = [CustID: int]


[CustID: int]

### (8):

In [13]:
summary4.show

+------+
|CustID|
+------+
+------+

